In [1]:
import glob, json, os
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def readJson(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data

def invTransform(extrinsics):
    assert extrinsics.shape == ((4,4))
    
    rotation = extrinsics[:3,:3]
    translation = extrinsics[:3,3].T
    
    invExtrinsics = np.identity(4)
    invExtrinsics[:3,:3] = rotation.T
    invExtrinsics[:3,3] = - rotation.T @ translation
    
    assert np.isclose(np.linalg.det(invExtrinsics[:3,:3]), 1.0)
    return invExtrinsics

def get_tf_cams(cam_dict, target_radius=1.):
    cam_centers = []
    for frame in cam_dict:
        C2W = np.array(frame['transform_matrix'])
        assert C2W.shape == ((4,4))
        cam_centers.append(C2W[:3, 3:4])

    def get_center_and_diag(cam_centers):
        cam_centers = np.hstack(cam_centers)
        avg_cam_center = np.mean(cam_centers, axis=1, keepdims=True)
        center = avg_cam_center
        dist = np.linalg.norm(cam_centers - center, axis=0, keepdims=True)
        diagonal = np.max(dist)
        return center.flatten(), diagonal

    center, diagonal = get_center_and_diag(cam_centers)
    radius = diagonal * 1.1

    translate = -center
    scale = target_radius / radius

    return translate, scale

def transform_pose(C2W, translate, scale):
        cam_center = C2W[:3, 3]
        C2W[:3, 3] = (cam_center + translate) * scale
        return C2W

In [3]:
# IRON wants normalized version
transforms = '/home/leyinghu/Documents/data/smplx_blender_addon_300_20220623/scene_female_kubric/transforms_test.json'
cam_dict = os.path.join(os.path.dirname(transforms), 'cam_dict_norm.json')

In [4]:
raw = readJson(transforms)

K = [
      raw['fl_x'],
      0.0,
      raw['cx'],
      0.0,
      0.0,
      raw['fl_y'],
      raw['cy'],
      0.0,
      0.0,
      0.0,
      1.0,
      0.0,
      0.0,
      0.0,
      0.0,
      1.0
    ]

img_size = [
      raw['w'],
      raw['h']
    ]

translate, scale = get_tf_cams(raw['frames'], target_radius=1.)

In [5]:
data = {}

for frame in raw['frames']:
    C2W = np.array(frame['transform_matrix'])
    assert C2W.shape == ((4,4))
    W2C = invTransform(C2W)
    W2C = transform_pose(W2C, translate, scale)
    assert np.isclose(np.linalg.det(W2C), 1.)
    
    data_frame = {}
    data_frame['K'] = K
    data_frame['W2C'] = W2C.flatten().tolist()
    data_frame['img_size'] = img_size
    
    data[frame['file_path']+'.png'] = data_frame

In [6]:
with open(cam_dict, 'w') as f:
    json.dump(data, f, indent=2, sort_keys=True)